In [44]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [45]:
def simulate_next_conditions(
    internal_temp, external_temp, internal_humidity, external_humidity, heating, venting
):
    temp = internal_temp
    humidity = internal_humidity

    heating_effect = 0.0
    if heating:
        temp += 1.5
        heating_effect = -0.5
    venting_effect = 0.0
    if venting:
        temp += 0.75 * (external_temp - temp)
        venting_effect = 0.3 * (external_humidity - internal_humidity)

    passive_gain = (
        0.05 * (100 - internal_humidity) if not venting and not heating else 0.0
    )

    humidity = internal_humidity + (heating_effect + venting_effect + passive_gain)
    humidity = max(0, min(100, humidity))
    temp += 0.05 * (external_temp - temp)

    return temp, humidity

In [46]:
n_samples = 1000
start_time = datetime.now()
datetimes = [start_time + timedelta(hours=1) for i in range(n_samples)]

external_temps = np.random.normal(loc=80, scale=40, size=n_samples)
external_humidies = np.random.uniform(50, 90, size=n_samples)

In [47]:
internal_temp = np.random.uniform(60, 80)
internal_humidity = np.random.uniform(60, 80)
data = []

for i in range(n_samples):
    ext_temp = external_temps[i]
    ext_hum = external_humidies[i]

    heating = int(internal_temp < 65)
    venting = int(internal_temp > 72)

    next_temp, next_hum = simulate_next_conditions(
        internal_temp, ext_temp, internal_humidity, ext_hum, heating, venting
    )

    data.append({
        "datetime": datetimes[i],
        "next_internal_temp": next_temp,
        "next_internal_hum": next_hum,
        "external_temp": ext_temp,
        "internal_temp": internal_temp,
        "external_humidity": ext_hum,
        "internal_humidity": internal_humidity,
        "heating": heating,
        "venting": venting,
    })

    internal_temp = next_temp
    internal_humidity = next_hum

In [48]:
df = pd.DataFrame(data)
df.to_csv("./data/synthetic_greenhouse_data.csv", index=False)

print("Generate synthetic data successfully.")
print(df.head())

Generate synthetic data successfully.
                    datetime  next_internal_temp  next_internal_hum  \
0 2025-07-12 01:10:24.721558           82.654585          74.623144   
1 2025-07-12 01:10:24.721558           42.161584          72.400116   
2 2025-07-12 01:10:24.721558           46.861082          71.900116   
3 2025-07-12 01:10:24.721558           48.971227          71.400116   
4 2025-07-12 01:10:24.721558           53.748413          70.900116   

   external_temp  internal_temp  external_humidity  internal_humidity  \
0      84.313560      77.328401          87.973196          68.901693   
1      29.549010      82.654585          67.213052          74.623144   
2     107.651543      42.161584          74.910098          72.400116   
3      60.563969      46.861082          51.643272          71.900116   
4     116.014946      48.971227          81.492902          71.400116   

   heating  venting  
0        0        1  
1        0        1  
2        1        0  
3       